In [41]:
import pandas as pd
from tqdm import tqdm
import time

In [42]:
# url = "https://race.netkeiba.com/race/result.html?race_id=202105010101"
# pd.read_html(url)[0]

In [43]:
pd.set_option('display.max_rows',20)
pd.options.display.width = 200
pd.options.display.column_space = 100

In [44]:
race_result = pd.read_pickle("df.pickle")
# race_result = pd.read_pickle("cleaned.pickle")
print(type(race_result.index[0]))

"202101010101" in race_result.index

<class 'str'>


True

In [45]:
race_id_lists = []

for 年 in range(2021,2022):
  for 競馬場 in range(1,11): # 1,11
    for 回 in range(1,5): # 1,11
      for 回目 in range(1,4): # 1,11
        for レース回 in range(1,3): # 1,13
          race_id = f"{年}{str(競馬場).zfill(2)}{str(回).zfill(2)}{str(回目).zfill(2)}{str(レース回).zfill(2)}"
          race_id_lists.append(race_id)
# print(race_id_lists)

# 実行時間 72分
def scrape_race_results(race_id_lists,par_race_results={}):
  race_result_dict = par_race_results
  for _race_id in tqdm(race_id_lists):
    if _race_id in race_result.index:
      continue
    try:
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      race_result_dict[_race_id] = pd.read_html(url)[0]
      # print(race_result_dict[race_id])
      time.sleep(1)
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return race_result_dict
  
add_race_result = scrape_race_results(race_id_lists)

100%|██████████| 240/240 [01:40<00:00,  2.38it/s]


In [46]:
len(add_race_result)

36

In [47]:
add_race_result

{'202104040101':     着順  枠  馬番         馬名  性齢    斤量   騎手     タイム     着差  人気  単勝オッズ   後3F コーナー通過順     厩舎  馬体重(増減)
 0    1  1   1   ベストリーガード  牡2  54.0  木幡巧  1:11.6    NaN   5    8.1  37.3     1-1    美浦牧   454(0)
 1    2  5   9   セイウンハルカニ  牡2  54.0   丸山  1:12.9      8   2    4.6  38.4     2-2   美浦竹内  460(+8)
 2    3  8  15   ハピネスガッサン  牝2  54.0  吉田豊  1:13.2      2   4    6.1  38.8     2-2   美浦田島  418(-6)
 3    4  8  14  フウジンショウジョ  牝2  54.0   三浦  1:13.7      3   1    4.1  39.0     4-4   美浦田島  474(-6)
 4    5  7  12       トレーネ  牝2  54.0   田辺  1:13.8    1/2   9   29.7  36.4   14-14   美浦佐藤  446(+8)
 5    6  5   8        カラニ  牡2  54.0   武藤  1:13.9    1/2  14  221.4  37.9     9-9   美浦水野  514(-8)
 6    7  3   4    ダイチスマイル  牡2  54.0  武士沢  1:14.2  1.1/2  12  185.9  37.7   13-13  美浦蛯名利  424(-6)
 7    8  3   5     ジーガーデン  牡2  54.0  内田博  1:14.2     クビ  11  108.3  38.0   11-11  美浦蛯名利  486(-6)
 8    9  4   7    トーセンコップ  牡2  54.0  戸崎圭  1:14.3     クビ   7   15.6  38.3     9-9  美浦古賀史  480(-2)
 9   10  6  11

In [48]:
for key in add_race_result.keys():
  add_race_result[key].index = [key] * len(add_race_result[key])
print(type(add_race_result))

<class 'dict'>


In [49]:
results = pd.concat([add_race_result[key] for key in add_race_result.keys()],sort=False)
print(type(results))
results

<class 'pandas.core.frame.DataFrame'>


,着順,枠,馬番,馬名,性齢,斤量,騎手,タイム,着差,人気,...,"(斤量, Unnamed: 5_level_1)","(騎手, Unnamed: 6_level_1)","(タイム, Unnamed: 7_level_1)","(着差, Unnamed: 8_level_1)","(人気, Unnamed: 9_level_1)","(単勝オッズ, Unnamed: 10_level_1)","(後3F, Unnamed: 11_level_1)","(コーナー通過順, Unnamed: 12_level_1)","(厩舎, Unnamed: 13_level_1)","(馬体重(増減), Unnamed: 14_level_1)"
202104040101,1,1.0,1,ベストリーガード,牡2,54.0,木幡巧,1:11.6,NaN,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202104040101,2,5.0,9,セイウンハルカニ,牡2,54.0,丸山,1:12.9,8,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202104040101,3,8.0,15,ハピネスガッサン,牝2,54.0,吉田豊,1:13.2,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202104040101,4,8.0,14,フウジンショウジョ,牝2,54.0,三浦,1:13.7,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202104040101,5,7.0,12,トレーネ,牝2,54.0,田辺,1:13.8,1/2,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202110040302,12,3.0,5,メイショウヤマツミ,牡3,56.0,小牧,1:52.0,9,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202110040302,13,3.0,6,サンデーインアスク,牝3,54.0,川田,1:52.1,1/2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202110040302,14,5.0,9,ベントラ,牡3,56.0,酒井,1:52.3,1.1/2,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202110040302,15,7.0,13,ギレルモ,牡3,56.0,岩田望,1:52.6,1.1/2,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
results.to_pickle("add.results.pickle")